# Illustrative example

In this example we will create an [acquisition server](https://openbci-stream.readthedocs.io/en/latest/notebooks/A3-server-based_acquisition.html) on a Raspberry Pi and stream EEG data in real-time from them through [Kafka](https://openbci-stream.readthedocs.io/en/latest/notebooks/02-kafka_configuration.html) using the [OpenBCI-Stream library](https://openbci-stream.readthedocs.io/en/latest/index.html).

<center>
<img src='images/illustrative.svg'></img>
</center>

Devices used:  

  * Raspberry Pi 4 Model B 4GB RAM  
  * Cyton Biosensing Board (8-channels)  
  * OpenBCI WiFi Shield  
  * Computer (with WiFi)  
  
Conventions used:  

  * The **red window frames** indicate that this window is "owned" by Raspberry through remote connection using SSH.  
  * The **IP** for the **Raspberry** is **192.168.1.1**  
  * The **IP** for the **OpenBCI WiFi Shield** is **192.168.1.113**  

## Acquisition server

The [guide to create an acquisition server](https://openbci-stream.readthedocs.io/en/latest/notebooks/A3-server-based_acquisition.html) explain the process to set up the server over a Raspberry Pi, after finish and reboot the system the Raspberry will be an **Acces Point**, we must connect the **OpenBCI WiFi Shield** to this network as well as the main computer (where BCI Framework will be executed).

We must verify that the respective daemons are running correctly on Raspberry:

    $ sudo systemctl status kafka zookeeper@kafka
<center>
<img src='images/illustrative/daemon1.gif'></img>
</center>

    $ sudo systemctl status stream_eeg stream_rpyc
<center>
<img src='images/illustrative/daemon2.gif'></img>
</center>

The system uses the NTP to synchronize clocks, so the Rasberry **must have a wired connection** to the internet to synchronize their own clock, to ensure this we can verify the connection and restart the daemon: 

    $ sudo systemctl restart ntpd
$ nptq -pn
<center>
<img src='images/illustrative/ntp1.gif'></img>
</center>

After a while, the clock will be synchronized (notice the * in the server **186.30.58.181**)
<center>
<img src='images/illustrative/ntp2.gif'></img>
</center>

We can verify the status of the **WiFi Shield** with the command:

    $ curl -s http:/192.168.1.113/board | jq
<center>
<img src='images/illustrative/shield.gif'></img>
</center>


The above commands being executed through a SSH connection, also can be done by connecting a monitor and a keyboard to the Raspberry.

## Configure montage

A simple montage 8 channels:

<center>
<img src='images/illustrative/montage.gif'></img>
</center>

## Configuration and connection with OpenBCI and then start the stream 

Connect with the Raspberry that is running under the IP **192.168.1.1** and the **WiFi Shield** on **192.168.1.113**. The sample frequency of **1000 samples per second** with a transmission of packages of **100 samples**.

<center>
<img src='images/illustrative/connection.gif'></img>
</center>

## Impedances

Once the streamming is started the impedances can be displayed from the **Montages tab**

<center>
<img src='images/illustrative/impedances.gif'></img>
</center>

## Raw EEG and topoplot

<center>
<img src='images/illustrative/raw.gif'></img>
</center>

## P300 speller

<center>
<img src='images/illustrative/p300.gif'></img>
</center>

## EEG records reading 

In [1]:
from openbci_stream.utils.hdf5 import HDF5Reader

filename = "record-04_20_21-13_58_25.h5"
file = HDF5Reader(filename)
print(file)
file.close()

record-04_20_21-13_58_25.h5
2021-04-20 13:58:25.717151
MARKERS: ['NO-TARGET', 'TARGET']
SAMPLE_RATE: 1000
STREAMING_SAMPLE_RATE: 100
DATETIME: 1618945105.717151
MONTAGE: standard_1020
CHANNELS: {1: 'Fp1', 2: 'Fp2', 3: 'T3', 4: 'C3', 5: 'C4', 6: 'T4', 7: 'O1', 8: 'O2'}
START-OFFSET: 346.3931083679199
SHAPE: [8, 68900]
END-OFFSET: 377.5792121887207


In [2]:
with HDF5Reader(filename) as file:
    data, classes = file.get_data(tmin=0, duration=0.125)

data.shape, classes.shape

((65, 8, 125), (65,))

## Online classifier

## P300 speller + Feedback